# Método SIMPLEX

In [60]:
from plotly import graph_objs as go
import plotly.offline as py
py.init_notebook_mode(connected=True)
import numpy as np

In [61]:
# Calcular zj:
def calcular_zj(c, base, A):
    return np.dot(c[base], A)

# Calcular zj-cj:
def calcular_zj_cj(zj, c):
     return zj - c

# Calcular zj-cj mínimo:
def minimo_zj_cj(zj_cj):
    return np.argmin(zj_cj)

# Calcular bk/Aij
def calcular_bk_Aij(bk, A, min_zjcj):
    return bk / A[:, min_zjcj]

# Buscar posición min bk_aij
def minimo_bk_Aij(bk_Aij):
    return np.argmin(bk_Aij) 

# Calcular costo total:
def calcular_costo_tot(c, base, bk):
    return np.dot(c[base], bk)

In [62]:
def visualizar_tabla(variables, base, cj, bk, Aij, zj, zj_cj, bk_aij, costo_total):
    Aij_t = list(map(list, zip(*Aij))) # Transponer
    
    # Construir columnas:
    header_values = ['', '', '$C_j$'] + cj
    col_i = ['$C_j base$'] + [cj[i] for i in base] + [''] + [costo_total]
    col_ii = ['$Var. Base$'] + [variables[i] for i in base]
    col_iii = ['$B_k$'] + bk + ['$z_j$', '$z_j - c_j$']
    rest_cols = [[variables[i]] + col + [zj[i]] + [zj_cj[i]] for i, col in enumerate(Aij_t)]
    
    cells_values = [col_i, col_ii, col_iii] + rest_cols + [['$Bk/A_{ij}$'] + bk_aij]
    
    # Construir y printear tabla:
    layout = go.Layout(margin=go.layout.Margin(l=25, r=25, b=0, t=25, pad=2), 
                       autosize=True)
    fig = go.Figure(data=[go.Table(header=dict(values=header_values),
                                   cells=dict(values=cells_values))],
                    layout=layout)
    
    fig.show()

In [63]:
# Input:
# variables = ['$X_1$', '$X_2$', '$X_3$', '$X_4$', '$X_5$']

# c = np.array([4, 4, 0, 0, 0]) # Coeficientes de funcion objetivo

# A = np.array([[ 6, 12,  1,  0,  0],
#               [12,  6,  0,  1,  0],
#               [ 9,  9,  0,  0,  1]]) # Coeficientes tecnológicos

# b = np.array([48000, 42000, 36000]) # Término LHS

# base = [2, 3, 4]

variables = ['$X_1$', '$X_2$', '$X_3$', '$X_4$', '$X_5$', '$X_6$']

c = np.array([1000, 1200, 0, 0, 0, 0]) # Coeficientes de funcion objetivo

A = np.array([[10,  5,  1,  0,  0,  0],
              [ 2,  3,  0,  1,  0,  0],
              [ 1,  0,  0,  0,  1,  0],
              [ 0,  1,  0,  0,  0,  1]]) # Coeficientes tecnológicos

b = np.array([200, 60, 34, 14]) # Término LHS

base = [2, 3, 4, 5]

iteraciones_simplex = 2


In [64]:
# Primera iteración:
zj = calcular_zj(c, base, A)

zj_cj = calcular_zj_cj(zj, c)

min_zjcj = minimo_zj_cj(zj_cj)

bk_Aij = calcular_bk_Aij(b, A, min_zjcj)

min_b = minimo_bk_Aij(bk_Aij)

costo_tot = calcular_costo_tot(c, base, b)

# Hacer print de la tabla:
visualizar_tabla(variables, 
                 base, 
                 c.tolist(), 
                 b.tolist(),
                 A.tolist(),
                 zj.tolist(),
                 zj_cj.tolist(),
                 bk_Aij.tolist(),
                 costo_tot)


c:\users\m_a_r\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning:

divide by zero encountered in true_divide



In [ ]:
# Elementos auxiliares
A_2 = np.zeros(A.shape)
b_2 = np.zeros(b.shape)

while any(zj_cj < 0):
    # Nueva base:
    base[min_b] = min_zjcj

    # Fila pivote:
    A_2[min_b, :] = A[min_b, :] / A[min_b, min_zjcj]
    b_2[min_b]    = b[min_b] / A[min_b, min_zjcj]
    

    # Recorrer filas excepto fila pivote:
    range_rows = set(range(0, A.shape[0])) - set([min_b]) 

    # Loop:
    for i in range_rows:
        # Nueva columna bk:
        b_2[i] = b[i] - b[min_b] * A[i, min_zjcj] / A[min_b, min_zjcj]
    
        for j in range(0, A.shape[1]):
            # Nueva matriz Aij:
            A_2[i,j] = A[i,j] - A[min_b, j] * A[i, min_zjcj] / A[min_b, min_zjcj]
            
    # Asignar a elementos principales:
    A = A_2.copy()
    b = b_2.copy()

    # Recalcular zj y zj_cj:
    zj = calcular_zj(c, base, A)
    
    zj_cj = calcular_zj_cj(zj, c)

    min_zjcj = minimo_zj_cj(zj_cj)

    bk_Aij = calcular_bk_Aij(b, A, min_zjcj)
    
    min_b = minimo_bk_Aij(bk_Aij)

    costo_tot = calcular_costo_tot(c, base, b)
    
    # Visualizar_tabla:
    visualizar_tabla(variables, 
                 base, 
                 c.tolist(), 
                 b_2.tolist(),
                 A_2.tolist(),
                 zj.tolist(),
                 zj_cj.tolist(),
                 bk_Aij.tolist(),
                 costo_tot)

c:\users\m_a_r\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning:

divide by zero encountered in true_divide

